In [43]:
import pandas as pd
import glob as glob
import datetime as datetime

In [44]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv

--2023-06-07 19:29:03--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2815 (2,7K) [application/json]
Saving to: 'bank_market_cap_1.json.9'

     0K ..                                                    100% 1,25G=0s

2023-06-07 19:29:05 (1,25 GB/s) - 'bank_market_cap_1.json.9' saved [2815/2815]

--2023-06-07 19:29:05--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Tra

In [45]:
log_file = "log_file.txt"
file_name = "exchange_rates.csv.3"
target_file = "bank_market_cap_gbp.csv"

In [46]:
def extract_json(file_name):
    dataframe = pd.read_json(file_name)
    return dataframe

In [47]:
def extract(file_name):

    extracted_data = pd.DataFrame(columns=["Name", "Market cap (US$ Billion)"])

    for json_file in glob.glob("*.json"):
        extracted_data = pd.concat([extracted_data, extract_json(json_file)], ignore_index = True)

    return extracted_data

In [48]:
def load_file(file_name):
    dataframe = pd.read_csv(file_name, index_col = 0)
    exchanged_rate = dataframe.loc["GBP"]
    return exchanged_rate




In [49]:
def transform(dataframe, exchanged_rate):
    transformed_data = dataframe.copy()
    transformed_data["Market Cap (US$ Billion)"] = transformed_data["Market Cap (US$ Billion)"] * exchanged_rate
    transformed_data["Market Cap (US$ Billion)"] = transformed_data["Market Cap (US$ Billion)"].round(3)
    transformed_data.rename(columns = {"Market Cap (US$ Billion)": "Market Cap (GBP£ Billion)"}, inplace= True)

    return transformed_data


In [50]:
def load(target_file, dataframe):
    dataframe = dataframe.to_csv(target_file, index = False)


In [51]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open(log_file, "w") as logfile:
        logfile.write(timestamp + ", " + message + "\n")


In [52]:
log("ETL started")

log("Extract phase started")
extracted_data = extract(file_name)
log("Extract phase ended")
extracted_data.head(5)

,Name,Market cap (US$ Billion),Market Cap (US$ Billion)
0,JPMorgan Chase,NaN,390.934
1,Industrial and Commercial Bank of China,NaN,345.214
2,Bank of America,NaN,325.331
3,Wells Fargo,NaN,308.013
4,China Construction Bank,NaN,257.399


In [53]:
log("Transform phase started")
transformed_data = transform(extracted_data, 0.7)
log("Transform phase ended")
transformed_data.head(5)

,Name,Market cap (US$ Billion),Market Cap (GBP£ Billion)
0,JPMorgan Chase,NaN,273.654
1,Industrial and Commercial Bank of China,NaN,241.650
2,Bank of America,NaN,227.732
3,Wells Fargo,NaN,215.609
4,China Construction Bank,NaN,180.179


In [54]:
log("Load phase started")
load(target_file, transformed_data)
log("Load phase ended")